In [1]:
import pyspark as ps
from pyspark.sql import SparkSession,SQLContext
from operator import add
from pyspark import SparkConf
#import findspark
#findspark.init()
import warnings

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.6:7077") \
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.executor.cores",3)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .appName("Group2")\
        .getOrCreate()
sc = spark_session.sparkContext
spark = SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/25 09:25:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/25 09:25:29 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/03/25 09:25:31 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [ ]:
df = spark.read.json("hdfs://192.168.2.6:9000/user/ubuntu/input/RC_2005-12")

22/03/25 09:23:15 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/03/25 09:23:30 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/03/25 09:23:45 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/03/25 09:24:00 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/03/25 09:24:15 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/03/25 09:24:30 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

In [ ]:
df.select("controversiality").distinct().collect()

In [ ]:
type(df)

In [ ]:
df.show(5)

In [ ]:
df_1 = df.select("body","controversiality")

In [ ]:
df_1.count()

In [ ]:
df_1.rdd.getNumPartitions()
df_1.repartition(100)

In [ ]:
(train_set, val_set, test_set) = df_1.randomSplit([0.98, 0.01, 0.01], seed = 2000)

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="body", outputCol="words")
hashtf = HashingTF(numFeatures=2**12, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "controversiality", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer, hashtf, idf, label_stringIdx])

pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
val_df = pipelineFit.transform(val_set)
train_df.show(5)

In [ ]:
from pyspark.ml.classification import LogisticRegression
import time

start = time.time()
lr = LogisticRegression(maxIter=100)
lrModel = lr.fit(train_df)
end = time.time()
print("time taken for training:{}".format(end-start))
predictions = lrModel.transform(val_df)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

In [ ]:
evaluator.getMetricName()

In [ ]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(val_set.count())
accuracy

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
x = [1,2,3,4]
y = [one,two,three,four]
plt.figure(figsize=(10, 8))
plt.plot(x,y)

plt.xlabel("number of cores")
plt.ylabel("Time taken (seconds)")
plt.savefig("strong_scaling.png")

In [2]:
import pyspark
from pyspark.sql import SparkSession
from operator import add
from pyspark import SparkConf

spark_session = SparkSession\
    .builder\
    .master("spark://192.168.2.6:7077") \
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
    .config("spark.shuffle.service.enabled", False)\
    .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
    .config("spark.executor.cores",1)\
    .config("spark.cores.max", 1)\
    .config("spark.driver.port",9998)\
    .config("spark.blockManager.port",10005)\
    .appName("Group2")\
    .getOrCreate()
sc = spark_session.sparkContext
spark = SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/25 09:27:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/25 09:27:52 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/03/25 09:27:54 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [3]:
df = spark.read.json("hdfs://192.168.2.6:9000/user/ubuntu/input/RC_2005-12")

22/03/25 09:28:25 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [4]:
df.take(5)

[Row(author='frjo', author_flair_css_class=None, author_flair_text=None, body='A look at Vietnam and Mexico exposes the myth of market liberalisation.', controversiality=0, created_utc=1134365188, distinguished=None, edited=False, gilded=0, id='c13', link_id='t3_17863', parent_id='t3_17863', retrieved_on=1473738411, score=2, stickied=False, subreddit='reddit.com', subreddit_id='t5_6', ups=2),
 Row(author='zse7zse', author_flair_css_class=None, author_flair_text=None, body='The site states "What can I use it for? Meeting notes, Reports, technical specs Sign-up sheets, proposals and much more...", just like any other new breeed of sites that want us to store everything we have on the web. And they even guarantee multiple levels of security and encryption etc. But what prevents these web site operators fom accessing and/or stealing Meeting notes, Reports, technical specs Sign-up sheets, proposals and much more, for competitive or personal gains...? I am pretty sure that most of them are h